### Tải dataset về

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY")
project = rf.workspace("postureexercise").project("sitting_posture-ozfmz")
version = project.version(16)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 65.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Sitting_posture-16 in yolov8:: 100%|██████████| 3400/3400 [00:00<00:00, 5504.61it/s]


In [ ]:
!nvidia-smi

!pip -q install --upgrade --index-url https://download.pytorch.org/whl/cu121 torch torchvision torchaudio

!pip -q install ultralytics

import torch, ultralytics
print("CUDA available:", torch.cuda.is_available(), "| torch:", torch.__version__)
ultralytics.checks()

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.5/112.6 GB disk)


### Train lần 1

In [ ]:
# ==================== PHA 1: TRAIN BASE ====================
from ultralytics import YOLO
model = YOLO("yolo11n-pose.pt")

results = model.train(
    data="/content/Sitting_posture-16/data.yaml",
    imgsz=960,
    epochs=50,
    batch=32,
    device=0,
    workers=4,
    cache='ram',
    freeze=4,
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.01,
    warmup_epochs=3,
    cos_lr=True,
    patience=20,

    # Augmentation
    mosaic=0.1, fliplr=0.0,
    degrees=15, translate=0.1, scale=0.7, shear=0.05,
    hsv_h=0.02, hsv_s=0.6, hsv_v=0.4,
    erasing=0.3,
    mixup=0.2, copy_paste=0.2,

    save_period=0, verbose=True
)

# ==================== PHA 2: FINE-TUNE TOÀN BỘ ====================
model2 = YOLO("/content/runs/pose/train/weights/best.pt")

results2 = model2.train(
    data="/content/Sitting_posture-16/data.yaml",
    imgsz=960,
    epochs=200,
    batch=18,
    device=0,
    workers=2,
    freeze=0,
    optimizer="AdamW",
    lr0=0.0005,
    weight_decay=0.01,
    warmup_epochs=2,
    cos_lr=True,
    patience=30,
    cache ='ram',

    # Augmentation
    mosaic=0.1, fliplr=0.0,
    degrees=15, translate=0.1, scale=0.7, shear=0.05,
    hsv_h=0.02, hsv_s=0.6, hsv_v=0.4,
    erasing=0.4,
    mixup=0.2, copy_paste=0.2,

    save_period=0, verbose=True
)

Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/Sitting_posture-16/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.3, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=4, half=False, hsv_h=0.02, hsv_s=0.6, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolo11n-pose.pt, momentum=0.937, mosaic=0.1, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0, plots=True, po

In [ ]:
model2.val(split='val', imgsz=960, device=0)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-pose summary (fused): 109 layers, 2,681,218 parameters, 0 gradients, 6.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1467.1±277.5 MB/s, size: 78.1 KB)
val: Scanning /content/Sitting_posture-15/valid/labels.cache... 220 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 220/220 153.8Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 14/14 1.4it/s 9.7s
                   all        220        221      0.735      0.825      0.835      0.496      0.735      0.825      0.833      0.772
              nga_phai         49         49      0.821      0.898      0.863      0.498      0.821      0.898      0.863       0.75
              nga_trai         51         51      0.939      0.922      0.956      0.588      0.939      0.922      0.956      0.888
          nghieng_phai         

ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aef882c8d70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,  

Upload kết quả lên drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/runs/pose /content/drive/MyDrive/pose_19_11